In [1]:
from __future__ import print_function
from tensorflow.keras.models import model_from_json, load_model
import numpy
import numpy as np
import time
import os
import math
import pyipopt
from numpy import *
import tensorflow as tf
import matplotlib.pyplot as plt

2024-03-04 14:49:11.474911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 14:49:11.603091: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 14:49:11.606926: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-04 14:49:11.606943: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:

################################### Simulation time step ##################################
delta = 0.01   # sampling time
hc = 1e-4     # integration time step
oper_time = 0.01  

################################## Initial states #########################################
CAi=-1.65
Ti=72
x1_nn=CAi
x2_nn=Ti
x1_record=[CAi]
x2_record=[Ti]
u1_record=[]
u2_record=[]

##################################### Constants ###########################################
a=1060
b=22
d=0.52  # Lyapunov function V=x^T*P*x

# CSTR PARAMETERS
F=5
V=1
k0=8460000
E=50000     # parametric drift #####E has the most effect on process dynamics######
R=8.314
T0=300
Dh=-11500
rho=1000
sigma=1000
cp=0.231
Qs=0
CA0s=4
w1_std=2.5  #disturbance std
w2_std=70   #disturbance std


########################################### steady-state ###################################
CAs= 1.9537
Ts=  401.8727

# state_ss=numpy.array([Ts, CAs])
# input_ss=numpy.array([Qs, CA0s])
state_ss=numpy.array([CAs,Ts])
input_ss=numpy.array([CA0s,Qs])

ROOT_FOLDER=os.getcwd()

##################################### MPC Parameters ######################################

NUM_MPC_ITERATION=10   # MPC TOTAL ITERATION


NUM_OUTPUTS = 2 # Number of state variables (RNN output)  
NUM_INPUTS = 4  # Number of RNN input
HORIZON = 2   ## MPC PREDICTION HORIZON (Depends on how many delta you want to predict with the RNN)

 
NUM_IN_SEQUENCE = 10  # Number of integration time steps actually used in MPC (equal to timestep of ML model)

NUM_MPC_INPUTS = 2*HORIZON  # Number of MPC inputs to be optimized, 2 is the number of control inputs 
NUM_MPC_CONSTRAINTS = HORIZON  # For each sampling time within prediction horizon, we have 1 constraint

realtime_data = None

setpoint=[0, 0]


In [3]:
# define scalers for both X and y
X_mean = np.load('X_mean.npy')
X_std = np.load('X_std.npy')
y_mean = np.load('y_mean.npy')
y_std = np.load('y_std.npy')

model = load_model('model.h5')
print(model)

x1_mean= X_mean[0]   # CA
x1_std= X_std[0]
x2_mean=X_mean[1]  # T
x2_std= X_std[1]
u1_mean= X_mean[2]    # CA0
u1_std=X_std[2]
u2_mean=X_mean[3]     # Q
u2_std=X_std[3]
y1_mean=y_mean[0]   # CA
y1_std=y_std[0]
y2_mean=y_mean[1]   # T
y2_std=y_std[1]

state_mean = np.array([x1_mean, x2_mean])  # [CA_input, T_input]
state_std = np.array([x1_std, x2_std])

input_mean = np.array([u1_mean, u2_mean])  # [CA0, Q]
input_std = np.array([u1_std, u2_std])

output_mean = np.array([y1_mean, y2_mean])  # [CA_output, T_output]
output_std = np.array([y1_std, y2_std])

2024-03-04 14:49:12.896807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-04 14:49:12.897072: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-04 14:49:12.897116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-03-04 14:49:12.897146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-03-04 14:49:12.899139: W tensorflow/c

In [4]:
def my_ens_prediction(num_horizon,my_rawdata,my_inputs):
    xx = []  
    nn_inputs = []  # NN input
    ensemble_output = numpy.zeros((num_horizon,NUM_OUTPUTS,NUM_IN_SEQUENCE))
    
    ensemble_output = ensemble_output.reshape(num_horizon,NUM_IN_SEQUENCE,NUM_OUTPUTS)
    x_test2 = my_rawdata[0:NUM_OUTPUTS].astype(float)
    x_test2= (x_test2-state_mean)/state_std           # my_rawdata is normal value; needs to normalize before feeding into NN

    predict_output_normal=[[0 for i in range(NUM_OUTPUTS)] for j in range(NUM_IN_SEQUENCE)]

    for i_model in range(num_horizon):    
  
        # ############################# get the normalized input for RNN #########################
         
        my_inputs_normalized = (my_inputs[2*i_model:2*(i_model+1)] - input_mean) / input_std    # my_inputs is also normal value; needs to normalize before feeding into NN
        xx = numpy.concatenate((x_test2,  my_inputs_normalized), axis=None).reshape((1, NUM_INPUTS))  # xx is the NN input
        xx = numpy.tile(xx, (NUM_IN_SEQUENCE, 1))  # duplicate #NUM_IN_SEQUENCE times

        nn_inputs = xx.reshape(1, NUM_IN_SEQUENCE, NUM_INPUTS) 
        # ############## use machine learning model to predict the next sampling time ##############
        
        predict_output = model(nn_inputs).numpy()
        predict_output = predict_output.reshape(NUM_IN_SEQUENCE, NUM_OUTPUTS)
        predict_output = predict_output * output_std + output_mean  # convert back to deviation form

        ####################### get the system state at the end of sampling time ################
        
        x_test2 = predict_output[-1, :]


        ###### transform the predicted states back to their original values in deviation form ######
        
        x_test2 = (x_test2 - state_mean) / state_std  # normalize input for next prediction

        ensemble_output[i_model,:,:] = predict_output[:, :] # store the predicted states 

        ########################################################################################


    return ensemble_output    


In [5]:
def eval_f(x):
    '''
    define the objective function
    '''
    assert len(x) == int(NUM_MPC_INPUTS)
    offset=0
    
    #### CALCULATE OUTLET CONC ###########
    df_ensemble_output = my_ens_prediction(num_horizon = HORIZON, my_rawdata=realtime_data, my_inputs=x)

    #### account for all intermediate steps ####
    for j in range (HORIZON):
        est_outlet_product = df_ensemble_output[j, :, 0:2]
        for i in range (int(NUM_IN_SEQUENCE)):  
           
             offset = offset + (setpoint[0] - (est_outlet_product[i, 1])) ** 2.0  + (setpoint[1] - (est_outlet_product[i, 0])) ** 2.0 * 1000



    return offset/100

def eval_grad_f(x):
    '''
    define the gradient of the objective function
    '''
    assert len(x) == int(NUM_MPC_INPUTS)
    step = 1e-1 # we just have a small step
    objp=objm=0
    grad_f = [0]*NUM_MPC_INPUTS
    xpstep = [0]*NUM_MPC_INPUTS
    xmstep = [0]*NUM_MPC_INPUTS
    for i_mpc_input in range(NUM_MPC_INPUTS):
        xpstep=x.copy()
        xmstep=x.copy()
        # for each variables, we need to evaluate the derivative of the function with respect to that variable, This is why we have the for loop
        xpstep[i_mpc_input]  = xpstep[i_mpc_input]+step 
        xmstep[i_mpc_input] = xmstep[i_mpc_input]-step

        # Evaluate the objective function at xpstep and xmstep
        objp=eval_f(xpstep) # This function returns the value of the objective function evaluated with the variable x[i] is perturebed +step
        objm=eval_f(xmstep) # This function returns the value of the objective function evaluated with the variable x[i] is perturebed -step
        #print ("obj ", objp, "   objm   ", objm)
        grad_f[i_mpc_input] = (objp - objm) / (2 * step) # This evaluates the gradient of the objetive function with repect to the optimization variable x[i]
    #print("Gradient: ", grad_f)
    return array(grad_f)


In [6]:
def eval_g(x):
    '''
    define the constraint function
    '''
    assert len(x) == int(NUM_MPC_INPUTS)

    CAd2=realtime_data[0]   ## current CA
    Td2=realtime_data[1]    ## current T
    g=array([-5.0]*NUM_MPC_CONSTRAINTS)   # g is the constraint (inequality) value; Initilize to be negative.


    if ((a*CAd2**2+d*Td2**2+2*b*CAd2*Td2-2)> 0): # (If V > \rho_min=2)
        
        df_ensemble_output3 = my_ens_prediction(num_horizon=1,my_rawdata=realtime_data, my_inputs=x)
        
        est_outlet = df_ensemble_output3[-1, -1, 0:2].reshape(1,2)
        
        # calculate the derivative of the Lyapunov function V(x,u) with respect to x
        dot_V1=(2*a * CAd2 + 2*b * Td2)*((est_outlet[0][0])-CAd2)/(0.01)+\
                (2*d*Td2 + 2*b * CAd2)*((est_outlet[0][1])-Td2)/(0.01)      
      

        '''
        This corresponds to the constraint: dot_V (x,u) < dot_V(x, \phi(x))
        To simplify calculation, we use the constraint, dot_V(x,u) < -15 V to make sure dot_V is negative.
        '''
        vv=a*CAd2**2+d*Td2**2+2*b*CAd2*Td2  # calculate the derivative of the Lyapunov function V(x,\phi(x))
        g[0]=dot_V1+15*abs(vv/100)   


    else:
        df_ensemble_output2 = my_ens_prediction(num_horizon=int(NUM_MPC_INPUTS / 2), my_rawdata=realtime_data,
                                               my_inputs=x)
           
        #####  only account for the last point #####
        for j in range(int(NUM_MPC_INPUTS / 2)):
            est_outlet_product2 = df_ensemble_output2[j, -1, 0:2] # only account for the last point
            
            g[j]= d * (est_outlet_product2[1]) ** 2+ 2 * b * (est_outlet_product2[1])*(est_outlet_product2[0]) + \
                  a*(est_outlet_product2[0]) ** 2 -2
            # this corresponds to the constraint: V(x,u) < 2 (\rho_min or \rho_nn in different papers)

    return  g

nnzj = NUM_MPC_CONSTRAINTS*NUM_MPC_INPUTS  ## number of nonzeros in the Jacobian of the constraint function


def eval_jac_g(x, flag):
    '''
    define the Jacobian of the constraint function
    '''
    
    if flag:
        list_x = []
        list_y=[]
        for i in range(int(NUM_MPC_INPUTS / 2)):
            list_x = list_x + [i] * NUM_MPC_INPUTS
            list_y = list_y +list(range(0, int(NUM_MPC_INPUTS)))

        return (array(list_x),
                array(list_y))


    else:
        assert len(x) == int(NUM_MPC_INPUTS)
        step = 1e-1 # we just have a small step
        gp=gm=numpy.zeros(NUM_MPC_CONSTRAINTS)
        xpstep=xmstep=numpy.zeros(NUM_MPC_INPUTS)
        jac_g = [[0]*int(NUM_MPC_INPUTS) for _ in range(NUM_MPC_CONSTRAINTS)]

        for i_mpc_input in range(NUM_MPC_INPUTS):
            xpstep=x.copy()
            xmstep=x.copy()
            # for each variables, we need to evaluate the derivative of the function with respect to that variable, This is why we have the for loop
            xpstep[i_mpc_input] += step 
            xmstep[i_mpc_input] -= step
            gp=eval_g(xpstep)
            gm=eval_g(xmstep)
            for num_constraint in range(NUM_MPC_CONSTRAINTS):
                jac_g[num_constraint][i_mpc_input] = (gp[num_constraint] - gm[num_constraint]) / (2 * step)
            #print ("in eval_jac_g_2:")
        return array(jac_g)

def apply_new(x):
    return True
def print_variable(variable_name, value):
    for i in range(len(value)):
        print("{} {}".format(variable_name + "["+str(i)+"] =", value[i]))


nnzh = NUM_MPC_INPUTS**2  ## number of nonzeros in the Hessian of the Lagrangian function

In [7]:
nvar = NUM_MPC_INPUTS   ## number of variables
x_lower=[0]* nvar   ## lower bound of the variables
x_upper=[0]* nvar  ## upper bound of the variables  
for i in range(int(HORIZON)):
    x_lower[2*i]= -3.5 
    x_lower[2 * i+1] = -5e5
    x_upper[2 * i] = 3.5
    x_upper[2 * i + 1] = 5e5
x_L = array(x_lower) #array([-3.5,-5e5])
x_U = array(x_upper) #array([3.5, 5e5])

### DEFINE THE UPPER BOUND AND LOWER BOUND OF THE CONSTRAINT ###
ncon = NUM_MPC_CONSTRAINTS      ## number of constraints
g_L = array([-2e19]*HORIZON)    ## lower bound of the constraints
g_U = array([0]*HORIZON)        ## upper bound of the constraints

print ("g_L", g_L, g_U)


g_L [-2.e+19 -2.e+19] [0 0]


In [8]:
for main_iteration in range(NUM_MPC_ITERATION):
    print ("Num Iteratin: ", main_iteration)

    rawdata = numpy.array([CAi, Ti])
    
    realtime_data=rawdata

    nlp = pyipopt.create(nvar, x_L, x_U, ncon, g_L, g_U, nnzj, nnzh, eval_f, eval_grad_f, eval_g, eval_jac_g)

    if main_iteration ==0 :
        x0 = array([0.0]*int(NUM_MPC_INPUTS))   # initial guess
      
    else:
        x0=x    # use the previous solution as the initial guess
        x0[0:-2]=x[2:]  # shift the previous solution to the left by 2
        x0[-2:]=x[-2:]  # keep the last two elements unchanged
        x_record=x

    """
    print x0
    print nvar, ncon, nnzj
    print x_L,  x_U
    print g_L, g_U
    print eval_f(x0)
    print eval_grad_f(x0)
    print eval_g(x0)
    a =  eval_jac_g(x0, True)
    print "a = ", a[1], a[0]
    print eval_jac_g(x0, False)
    print eval_h(x0, pi0, 1.0, False)
    print eval_h(x0, pi0, 1.0, True)
    """

    """ You CAd2 set Ipopt options by calling nlp.num_option, nlp.str_option
    or nlp.int_option. For instance, to set the tolarance by calling

        nlp.num_option('tol', 1e-8)

    For a complete list of Ipopt options, refer to

        http://www.coin-or.org/Ipopt/documentation/node59.html

    Note that Ipopt distinguishs between Int, Num, and Str options, yet sometimes
    does not explicitly tell you which option is which.  If you are not sure about
    the option's type, just try it in PyIpopt.  If you try to set one type of
    option using the wrong function, Pyipopt will remind you of it. """

    nlp.int_option('max_iter', 1000)    # maximum number of iterations
    nlp.num_option('tol', 1e-5)         # convergence tolerance
    nlp.int_option('print_level', 2)    # print out the process
    print("Going to call solve")        # solve the problem
    print("x0 = {}".format(x0))         # initial guess
    x, zl, zu, constraint_multipliers, obj, status = nlp.solve(x0)  

    nlp.close()

    print("Solution of the primal variables, x")
    print_variable("x", x)
    print ("status=", status)

    print("Objective value")
    print("f(x*) = {}".format(obj))
    print ("Control action=:  ", x[0], x[1])

    x1=CAi  
    x2=Ti  

    # w is the disturbance
    # w1 =numpy.random.normal(0, w1_std, 1)
    # w2 =numpy.random.normal(0, w2_std, 1)
    # if w1>w1_std:
    #     w1=w1_std
    # if w1<-w1_std:
    #     w1=-w1_std
    # if w2>w2_std:
    #     w2=w2_std
    # if w2>w2_std:
    #     w2=w2_std
    #print (numpy.asscalar(w1))
    #print (numpy.asscalar(w2))
    for kk in range (int(delta/hc)):    # apply the control action for real model


        x1_new = x1 + hc * ((F / V) * (x[0] - x1) -
                            k0 * ((numpy.exp(-E / (R * (x2 + Ts)))*(x1 + CAs) * (x1 + CAs))
                                  - numpy.exp(-E / (R * Ts)) * CAs * CAs))

        x2_new = x2 + hc * (((F / V) * (-x2) + (-Dh / (sigma * cp)) *
                             (k0 * ((numpy.exp(-E / (R * (x2 + Ts))) * (x1 + CAs) * (x1 + CAs)) -
                                      numpy.exp(-E / (R * Ts)) * CAs * CAs)) + (x[1] / (sigma * cp * V))))



        x1 = x1_new
        x2 = x2_new

        # if (kk%5==1):
        #     x1_record.append(x1)
        #     x2_record.append(x2)
        #     u1_record.append(x[1])
        #     u2_record.append(x[0])

    CAi=x1
    Ti=x2




    print('Real model output x1 x2 in deviation form:   ', x1, x2)

    x1_record.append(x1)
    x2_record.append(x2)
    u1_record.append(x[0])
    u2_record.append(x[1])

print ("x1_record: ",x1_record)
print ("x2_record: ",x2_record)

print ("u1_record: ",u1_record)
print ("u2_record: ",u2_record)

# numpy.savetxt("x1.txt",   x1_record, fmt="%f",  delimiter=" ")
# numpy.savetxt("x2.txt",   x2_record, fmt="%f",  delimiter=" ")


# numpy.savetxt("u1.txt",   u1_record, fmt="%f",  delimiter=" ")
# numpy.savetxt("u2.txt",   u2_record, fmt="%f",  delimiter=" ")



Num Iteratin:  0
[PyIPOPT] Ipopt will use Hessian approximation.

[PyIPOPT] Problem created
Going to call solve
x0 = [0. 0. 0. 0.]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Solution of the primal variables, x
x[0] = 3.5000000349985347
x[1] = -0.8601783386446912
x[2] = 3.5000000349975773
x[3] = -0.9449021993056471
status= -1
Objective value
f(x*) = 1251.1258619520822
Control action=:   3.5000000349985347 -0.8601783386446912
Real model output x1 x2 in deviation form:    -1.3491245181524205 66.01608363831458
Num Iteratin:  1
[PyIPOPT] Ipopt will use Hessian approximation.

[PyIPOPT] Problem created
Going to call solve
x0 = [ 3.50000003 -0.9449022   3